# Del 4: Sampling og den diskrete Fouriertransformasjonen

Fouriertransformasjonen vi har sett på hittil er det som kalles den **kontinuerlige** Fouriertransformasjonen, og er definert for kontinuerlige funksjoner. Dette skaper problemer når vi skal implementere Fouriertransformasjonen på en datamaskin. Datamaskiner kan kun gjennomføre regneoperasjoner på diskrete verider, og vi må derfor ha en diskret versjon av Fouriertransformasjonen som gir en diskret resultat.

Den diskrete Fouriertransformasjonen (**DFT**) er gitt ved følgende formel:

$$X(k) = \sum_{n=0}^{N - 1} x(n)e^{-i\frac{2\pi nk}{N}}$$

La oss se hvordan vi kan gå fra den kontinuerlige Fouriertransformasjonen til den diskrete. Altså fra:

$$X(\xi) = \int_{-\infty}^{\infty} f(t)e^{-i 2\pi \xi t} dt$$

til DFT-en. 

Vi gjør dette i to steg. Først bytter vi ut den kontinuerlige funksjonen $f(t)$ med en diskret funksjon $x(n)$. I vår tilfelle er det tilskrekkelig å se på denne funksjonen som en tallfølge (ofte representert som en array i Python). Vi må også benytte oss av summetegnet i stedet for integraltenget, siden vi nå har en diskret funksjon.

Fouriertransformasjonen vår ser nå slik ut: 

$$X(\xi) = \sum_{-\infty}^{\infty} x(n)e^{-i 2\pi \xi n}$$

Dette kalles for **DTFT-en**, eller diskret-tids Fouriertransformasjonen. Merk at $\xi$, altså variablen til frekvens, er fortsatt en **kontinuerlig** variabel. Så selv om DTFT-en gjennomfører Fouriertransormasjonen på en diskret funksjon, gir den fortsatt en kontinuerlig resultat. I tillegg går summegrensene fra og til uendeligheten, som er litt rart når vi skal jobbe med endelige tallfølger på en datamaskin. Vi må altså diskretisere frekvensvariabelen $\xi$, og endre på summegrensene for å få det vi vil. 

Vi diskretiserer $\xi$ på følgende måte: 

$$\xi = \frac{k}{N}$$

hvor $k$ er den k-te "bin-en," eller frequency bin, i $X(k)$ og $N$ er antall punkter i $x(n)$. Ved å diskretisere $\xi$, har vi grovt sagt "kuttet opp" frekvensvariabelen $\xi$ inn i $N$ deler. En slik del av frekvensvariabelen kalles ofte for en "bin," og vi har altså $N$ slike binner.

Vi endrer også summegrensene slik at de summerer over en endelig lengde, altså fra første element i $x(n)$ til den siste. Fouriertransformasjonen vår blir da:

$$X(k) = \sum_{n=0}^{N - 1} x(n)e^{-i\frac{2\pi nk}{N}}$$

Hvor $x(n)$ er en diskret funksjon, og $X(k)$ er den diskrete Fouriertransformasjonen av $x(n)$. $N$ er antall punkter i $x(n)$, og $k$ er den k-te frekvensen. Dette kalles for DFT-en, eller den diskrete Fouriertransformasjonen, og det er akkurat denne vi bruker når vi implementerer Fouriertransformasjonen på en datamaskin. Legg merke til at vi har samme antall binner, altså frekvenser som antall punkter i $x(n)$, altså $N$ binner.

Prinsippet til DFT-en er da akkurat den samme som med den kontinuerlige Fouriertransformasjonen: vi tar en eller annen funksjon som befinner seg i tidsdomenet, og transformerer den til frekvensdomenet. Én ting som er verdt å bemerke seg er at verdien på frekvenskomponentene i DFT-en er avhengig av antall punkter i $x(n)$. Kort fortalt, så kommer ikke enhetene til frekvenskomponentene i DFT-en til å samsvare med de i tidssignalet. Dette løses ved å normalisere DFT-en: 

$$X(k)/N$$

Da vil verdien til frekvenskomponentene tilsvare det vi faktisk har i tidssignalet.

Allikevel er det noen nyanser med DFT-en som vi må være klar over. For at vi skal kunne forstå oss på disse nyansene, må vi først se på hvordan man diskretiserer en kontinuerlig funksjon, også kalt sampling:

## Sampling



Sampling er når vi tar en kontinuerlig funksjon og gjør den diskret. Intuitivt kan vi si at vi deler opp den kontinuerlige funksjonen i punkter med lik intervall mellom seg, og så tar vi verdien til funksjonen i disse punktene: 

<center><img src="plots/p4_cont.png" width="200px"> <img src="plots/p4_cont_samp.png" width="200px"> <img src="plots/p4_samp.png" width="200px"></center>

Etter vi har gjort dette har vi en diskret tallfølge som kan lagres på og bli behandlet av en datamaskin. 

La oss introdusere noen nye begreper: 

- $N$: Total antall samplepunkter, altså lengden på $x(n)$. 
- $T$: Samplingsintervall. Også kjent som samplingsperiode. Dette er avstanden mellom hvert samplepunkt, med sekunder som enhet.
- $f_s$: Samplingsfrekvens. Dette er antall samplepunkter per sekund, med enhet Hertz (Hz). Samplingsfrekvensen er den inverse av samplingsintervallet: $f_s = \frac{1}{T_s}$

Hvis vi tar for oss en sinusfunksjon i Python:
```python
t = np.linspace(0, 10, 1000) # tid i sekunder, fra 0 til 10 sekunder med 1000 samplepunkter
sinus = np.sin(t)
```

er $N = 1000$, samplingsintervallet er $T_s = 10/1000 = 0.01$ sekunder, og samplingsfrekvensen er $f_s = 1/T_s = 100$ Hz.

Konverteringen fra kontinuerlig til diskret er en prosess som kalles for analog til digital konvertering, og er noe som skjer på hardwarenivå, for eksempel i en mikrofon som tar inn en lydbølge (en analog / kontinuerlig verdi) og konverterer den til en digital tallfølge ved å sample den. I kodesnutten ovenfor definerer vi arrayet $t$ og bruker den som input inn i Numpy sin sinusfunksjon, som gir en array med verdiene til sinusfunksjonen i de punktene. Vi har da ikke tatt en analog signal og konvertert den til en digital en, men heller laget en diskret tallfølge fra bunnen av.


## Undersampling og Nyquists samplingsteorem

Hva skjer hvis vi har en kontinuerlig funksjon og hakker den opp (sampler) den med en for lav samplingsfrekvens? La oss ta sinusfunksjonen fra kodesnutten ovenfor som en eksempel:

```python
t = np.linspace(0, 10, 1000) 
sinus = np.sin(t)
```

Hvis vi plotter den rett ut ser den slik ut:

<center><img src="plots/p4_sine.png" width="400px"></center>

Ser ut som en helt vanlig sinusfunksjon. La oss nå minke samplingsfrekvensen, altså øke samplingsintervallet til noe banalt: 

```python
t = np.linspace(0, 10, 3) 
sinus = np.sin(t)
```

Vi har altså en sinusfunksjon med en frekvens på $2\pi$ Hz, som vi skal prøve å sample med kun tre punkter fordelt på 10 sekunder. Samplingsintervallet er nå $T_s = 10/3$ sekunder, og samplingsfrekvensen er $f_s = 1/T_s = 3/10$ Hz. Hvis vi plotter denne funksjonen så får vi:

<center><img src="plots/p4_sine_usamp.png" width="400px"></center>

Her kan vi bemerke at grafen er særdeles stygg for å være en sinusfunksjon, og at vi mister informasjon om hvor ofte funksjonen svinger. I det "fine" plottet ser vi tydelig at funksjonen svinger ca. 1.5 ganger i løpet av 10 sekunder, mens ut ifra det stygge plottet så kan vi fort tro at den svinger mindre enn en gang i løpet av samme tid.

Dette kalles for **undersampling**. Vi har altså en funksjon med en eller annen frekvenskomponent, i dette tilfellet én frekvenskomponent med frekvens lik $2\pi$ Hz, og vi sampler den med en samplingsfrekvens som er for lav. Dette fører til at vi mister informasjon om frekvenskomponentene til funksjonen. 

Men hva er "for lav" samplingsfrekvens? Altså, hva er den laveste samplingsfrekvensen vi kan ha før vi begynner å miste informasjon om funksjonen? Da er vi inne på **Nyquists samplingsteorem**, som sier at for å unngå undersampling, så **må samplingsfrekvensen være minst dobbelt så stor som den høyeste frekvenskomponenten i funksjonen**. Den høyeste frekvensen vi kan sample uten å miste informasjon kalles for **Nyquist-frekvensen**, som er da halvparten av samplingsfrekvensen.

## Konsekvenser av diskretisering og undersampling i frekvensdomenet 

Så langt har vi sett på sampling og undersampling i tidsdomenet. Men har undersampling, og det at vi i det hele tatt ser på en diskret funksjon, konsekvenser i frekvensdomenet? Svaret er ja, og det er akkurat dette som skiller resultatet til DFT-en fra den kontinuerlige Fouriertransformasjonen. 

### Periodisering og "falske" frekvenser

En konsekvens av å diskretisere Fouriertransformasjonen at DFT-en "ser" funksjonen den skal transformere som en periodisk funksjon, uansett om den faktisk er det eller ikke. I tillegg til dette så kommer frekvenspektret til å være periodisk, som er et konsekvens av at vi har transformert en diskret, og ikke kontinuerlig, funksjon:

<center><img src="plots/cont_disc_spectra.gif" width="800px"></center>

*Forholdet mellom den kontinuerlige Fouriertransformasjonen og den diskrete Fouriertransformasjonen. Fra* *[Wikipedia](https://en.wikipedia.org/wiki/Discrete_Fourier_transform)* *, av bruker Sbyrnes321.*

*Venstre kolonne: En kontinuerlig, aperiodisk funksjon i tidsdomenet (øverst) og dens Fouriertransformasjon (nederst).* 

*Midt-venstre kolonne: En kontinuerlig, periodisk funksjon i tidsdomenet (øverst). Fouriertransformasjonen (nederst) er null unntatt ved diskrete punkter, altså den består av en endelig antall frekvenskomponenter.*

*Midt-høyre kolonne: Den kontinuerlige, aperiodiske funksjonen er diskretisert (øverst). Vi tar dens Fouriertransformasjon med DTFT-en (ikke DFT-en!), og dens Fouriertransformasjon (nederst) er en repetisjon av frekvensspektret av dens kontinuerlige versjon.*

*Høyre kolonne: Vi tar DTFT-en til den forrige funksjonen og diskretiserer den, og dette blir da en DFT (nederst). I DFT-en sin tilfelle blir funksjonen i tidsdomenet en periodisk repetisjon av den opprinnelige diskrete funksjonen (øverst). Dette er hva DFT-en "ser" når den transformerer en funksjon til frekvensdomenet.*

Oppsumert: selv om en kontinuerlig funksjon er i utgangspunktet aperiodisk og har et aperiodisk frekvensspekter, kommer dens frekvensspekter til å bli periodisk om vi diskretiserer funksjonen og transformerer den med DFT-en. Kort fortalt så er dette fordi i motsetning til DTFT-en, som summerer fra og til uendeligheten, summerer DFT-en kun over en endelig lengde, og dermed må den anta at funksjonen er periodisk. Man kan med andre ord si det slik: det "virkelige" frekvensespekteret kommer til å bli kopiert og plassert med jevne mellomrom i frekvensdomenet:

<center><img src="plots/p4_fakes.png" width="600px"></center>

Dette er et eksempel på hvordan frekvensspektret kommer til å se etter vi har tatt DFT-en av en funksjon. Kun de "ekte" frekvenskomponentene er til stede i den opprinnelige funksjonen, mens de falske frekvenskomponentene er en konsekvens av å bruke DFT-en. Avstanden mellom er lik samplingsfrekvensen.

Så hva skjer når vi undersampler, og avstanden mellom hver periode i frekvensdomenet blir mindre enn den høyeste virkelige frekvenskomponenten? Da vil vi få noe som kalles for aliasing.

### Aliasing

Aliasing skjer når vi undersampler en signal, og de falske frekvenskomponentene begynner å overlappe med de ekte. Som følge får vi villedende informasjon om hvilke frekvenskomponenter inngår i funksjonen:

<center><img src="plots/p4_alias.png" width="600px"></center>

I eksemplet over er samplingsfrekvensen mindre enn det dobbelte av den høyeste frekvenskomponenten i funksjonen. Resultatet er at de falske frekvenskomponentene overlapper med de ekte, som gir oss feil informasjon om frekvensinnholdet til funksjonen, ergo aliasing. Grafisk sett kan vi legge merke til at de "ekte" frekvenskomponentene som befinner seg over Nyquist-frekvensen blir speilet over Nyquist-frekvensen. For å unngå aliasing må vi altså sørge for at samplingsfrekvensen er minst dobbelt så stor som den høyeste frekvenskomponenten i funksjonen, og da er vi tilbake på Nyquists samplingsteorem.

Det er viktig å huske at aliasing, undersampling, og periodiseringen av frekvensspektret skjer kun når vi bruker DFT-en! Både DTFT-en og den kontinuerlige Fouriertransformasjonen slipper unna disse problemene, og gir oss et korrekt bilde av frekvensinnholdet til funksjonen. Allikevel så har vi ikke noe valg enn å bruke DFT-en når vi skal implementere Fouriertransformasjonen på en datamaskin, så disse nyansene er noe vi må være klar over når vi jobber med DFT-en.

## Spektrallekkasje

En annen konsekvens av å diskretisere Fouriertransformasjonen er noe som kalles for spektrallekkasje. Kort fortalt er spektrallekkasje når frekvenskomponentene i DFT-en "lekker" inn i andre frekvenskomponenter, og vi får et spektrum som ikke samsvarer med det faktiske frekvensinnholdet til funksjonen (dette er åpenbart noe vi vil unngå). Spektrallekkasje og hvordan man minker den er et stort tema i seg selv, og vi skal ikke gå i dybden på det. Allikevel er det meget nyttig å vite følgende: 

Om du har en periodisk funksjon representert i en tidsserie med en lengde ikke er et heltallig multiplum av perioden til funksjonen, vil du få spektrallekkasje. Dette innebærer at frekvenskomponentene i DFT-en blir "lekket" inn i andre frekvenskomponenter, og vi får et spektrum som ikke samsvarer med det faktiske frekvensinnholdet til funksjonen. 

La oss ta for oss følgende eksempel: Vi har en sinus med frekvens 1 Hz, representert med to tidsserier: en som varer i 3 sekunder, og en annen en som varer i 2.35 sekunder: 

<center><img src="plots/p4_sine_full.png" width="400px"> <img src="plots/p4_sine_leak.png" width="400px"></center>

Legg merke til at sinusen i det første plottet har tre hele perioder i tidsserien, mens sinuskurven i det andre plottet har en periode som er "kuttet av." Vi tar DFT-en av begge tidsseriene, og får følgende spektra:

<center><img src="plots/p4_fft_full.png" width="400px"> <img src="plots/p4_fft_leak.png" width="400px"></center>

Her ser vi at spektrallekkasje har skjedd i det andre spekteret. Frekvenskomponenten på 1 Hz har "lekket" inn i andre frekvenskomponenter, og vi får et spektrum som ikke samsvarer med det faktiske frekvensinnholdet til funksjonen. 

Spektrallekkasje er et problem som er vanskelig å unngå i praksis, ettersom det er ikke alltid vi vet frekvensinnholdet til signalet på forhånd, som gjør det vanskelig å tilpasse lengden på datasettet til frekvensinnholdet. Allikevel er det noen metoder som kan brukes for å minimere spektrallekkasje, som for eksempel vinduing. Dette er et stort tema i seg selv, og vi skal ikke gå i dybden på det her, men det er noe som er verdt å vite om skulle du støte på spektrallekkasje i praksis.

## Litt om FFT-en 

Når man implementerer Fouriertransformasjonen, altså DFT-en, på en datamaskin, bruker vi ofte begrepet FFT. FFT står for fast Fourier transform (rask Fouriertransformasjon) og er en algoritme som regner ut DFT-en på en særdeles rask måte, mye raskere om vi hadde implementert DFT-en rett fram. **Resultatet av en FFT er akkurat den samme som en DFT.** Ikke bli forvirret av at dette er to forskjellige begreper, de regner ut akkurat det samme. Den eneste forskjellen er at FFTen er en optimalisering av DFT formelen vi så tidligere. 

## Oppgaver 

**Oppgave 1: De ulike transformasjonene**

a) Funksjonene vi transformerer, samt frekvensvariablene til frekvensdomenene er enten kontinuerlige eller diskrete avhengig av hvilken transformasjon vi bruker. Hvis vi tar i betraktning den kontinuerlige Fouriertransformasjonen, DTFT-en, og DFT-en, hvilke av disse transformerer kontinuerlige funksjoner og hvilke transformerer diskrete funksjoner? Hvilke av disse har kontinuerlige frekvensvariabler og hvilke har diskrete frekvensvariabler?

b) Av de tre transformasjonene, hvilke(n) kan bli regnet ut på en datamaskin? Forklar hvorfor.

**Oppgave 2: sampling og Nyquist-frekvensen**

a) Se for deg en funksjon som er en sum av to sinusfunksjoner med frekvenser 1 Hz og 2 Hz. Hva er den laveste samplingsfrekvensen vi kan ha før vi begynner å miste informasjon om funksjonen?

b) Du sampler en funksjon med en samplingsfrekvens på 10 Hz. Hva er Nyquist-frekvensen?

**Oppgave 3: Aliasing**

a) Du har en funksjon som er en sum av to sinusfunksjoner med frekvenser 100 Hz og 300 Hz. Du undersampler funksjonen med en samplingsfrekvens på 450 Hz. Gitt samplingsfrekvensen, hva er den laveste frekvensen du kan sample (Nyquist-frekvensen) før du får villedende informasjon om frekvenssinnholdet til signalet? 

b) Om hvilken frekvens vil den høyeste frekvenskomponenten i funksjonen bli speilet over Nyquist-frekvensen? Hvor i frekvensspektret vil den speilede frekvenskomponenten ligge? Vil komponenten på 100 Hz bli speilet over Nyquist-frekvensen?


## *Fasit*
1a) Den kontinuerlige Fouriertransformasjonen: kontinuerlig funksjon, kontinuerlig frekvensvariabel. 
DTFT-en: diskret funksjon, kontinuerlig frekvensvariabel. 
DFT-en: diskret funksjon, diskret frekvensvariabel.

1b) DFT-en er den eneste av de tre som kan regnes ut på en datamaskin. Dette er fordi både funksjonen DFT-en transformerer og frekvensvariabelen er diskrete, og en datamaskin kan dermed representere dem som tallfølger. Datamaskiner kan ikke regne ut kontinuerlige funksjoner eller variabler, og dermed kan ikke den kontinuerlige Fouriertransformasjonen eller DTFT-en regnes ut på en datamaskin.

2a) Den største frekvenskomponenten i funksjonen er 2 Hz, og vi må derfor sample med en samplingsfrekvens som er minst dobbelt så stor som 2 Hz, altså 4 Hz. Om samplingsfrekvensen er lavere enn 4 Hz, vil vi miste informasjon om frekvensinnholdet til funksjonen.

2b) Nyquist-frekvensen er halvparten av samplingsfrekvensen, altså 5 Hz. Det vil si at den høyeste frekvensen vi kan sample uten å miste informasjon er 5 Hz

3a) Nyquist-frekvensen er halvparten av samplingsfrekvensen, altså 225 Hz. Det vil si at den laveste frekvensen vi kan sample før vi begynner å miste informasjon er 225 Hz.

3b) Den høyeste frekvenskomponenten i funksjonen er på 300 Hz, og den vil bli speilet over Nyquist-frekvensen, som er 225 Hz. Den speilede frekvenskomponenten vil da ligge på $\xi_{alias} = \xi_{speil} - (\xi - \xi_{speil}) =225 - (300 - 225) = 150[Hz]$. Komponenten på 100 Hz vil ikke bli speilet over Nyquist-frekvensen, da den er lavere enn Nyquist-frekvensen.